In [ ]:
%pip install openai zeroentropy

# Setting up ZeroEntropy

First, we setup ZeroEntropy with a test document.

This will require an API Key from https://dashboard.zeroentropy.dev/api-keys

In [ ]:
from zeroentropy import ZeroEntropy, ConflictError
import time
import os

# Index an example collection into ZeroEntropy
# Go to https://dashboard.zeroentropy.dev/api-keys if you need an API Key!
zclient = ZeroEntropy(api_key=os.getenv("ZEROENTROPY_API_KEY"))

COLLECTION_NAME = "deepresearch-test"
PATH = "secret.txt"
try:
    zclient.collections.add(collection_name=COLLECTION_NAME)
    print("Collection Created!")
except ConflictError:
    print("Collection already exists!")

# Add the document
try:
    zclient.documents.add(
        collection_name=COLLECTION_NAME,
        path=PATH,
        content={
            "type": "text",
            "text": "The Deep Research secret is: \"489284\"",
        }
    )
    print("Document Added!")
except ConflictError:
    print("Document already exists!")

# Wait until it's indexed
while True:
    if zclient.documents.get_info(collection_name=COLLECTION_NAME, path=PATH).document.index_status == "indexed":
        break
    time.sleep(1)
print("The document is indexed!")

Collection already exists!
Document already exists!
The document is indexed!


# Calling OpenAI DeepResearch

Now we can simply call OpenAI using our OpenAI API Key.

This code will pass in ZeroEntropy's MCP url, and also our `zclient`'s API Key and collection name.

Then, OpenAI Deep Research will find the secret and output it!

In [ ]:
from openai import OpenAI

# Using OpenAI API Key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.responses.create(
  model="o4-mini-deep-research",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": "You are a research assistant that searches MCP servers to find answers to your questions.",
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "In one search call, find the secret",
        }
      ]
    }
  ],
  reasoning={
    "summary": "auto",
  },
  tools=[
    {
      "type": "mcp",
      "server_label": "ZeroEntropy",
      "server_url": "https://openai-deepresearch.zeroentropy.dev/sse/",
      "headers": {
        "Authorization": f"Bearer {zclient.api_key}",
        "X-Collection-Name": COLLECTION_NAME,
      },
      "allowed_tools": [
        "search",
        "fetch",
      ],
      "require_approval": "never",
    }
  ]
)

print(response.output_text)

# Secret discovered with one search

- **Search query:** Performed one search call on the ZeroEntropy data with the query `"the secret"`.  
- **Search result:** The query returned a file named **`secret.txt`**.  
- **Secret content:** Opening `secret.txt` shows the line:  
  > The Deep Research secret is: `"489284"` ([api.zeroentropy.dev](https://api.zeroentropy.dev/v1/documents/get-document?token=cRR5jBh9-eyNMng8nH0t3r_pvCTkxQpuJEArf2YnSyvLk7rPcJWzOlrIzjYGp8raesi-8Sn8XuxDcTpfjkChpkI_spOxNUjdh8iZofZJSKfzr4x50rZbukUutEiJa_D8oD_MXFT_3FB80zB_3EojRdTyer1-oHWZoCEbV5gWNfsYLdRjR11QNEREsIFjSxF84Q5g6VQI9S9LD4qc9Q3Mkt2xn7Qlv0Gx8FsZTbFlUmjT7qUtO9Ue6XgiZrOt9Duj)).  
  This line clearly reveals the secret number.  
- **Answer:** The secret is **489284** ([api.zeroentropy.dev](https://api.zeroentropy.dev/v1/documents/get-document?token=cRR5jBh9-eyNMng8nH0t3r_pvCTkxQpuJEArf2YnSyvLk7rPcJWzOlrIzjYGp8raesi-8Sn8XuxDcTpfjkChpkI_spOxNUjdh8iZofZJSKfzr4x50rZbukUutEiJa_D8oD_MXFT_3FB80zB_3EojRdTyer1-oHWZoCEbV5